In [121]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from functools import wraps
from sklearn.metrics import ndcg_score
from glob import glob
from os import path as osp

In [122]:
BASE_DIR = 'race_data/'
USE_COL = ['着順', '馬名', '性齢', '斤量', '騎手', 'タイム', '着差', '単勝', '人気', '馬体重', '調教師', 'horse_id', 'jockey_id']

In [123]:
p_race_csvs = glob(osp.join(BASE_DIR, 'horse_info_*.csv'))
p_race_csvs

['race_data\\horse_info_202408010101.csv',
 'race_data\\horse_info_202408010102.csv',
 'race_data\\horse_info_202408010103.csv',
 'race_data\\horse_info_202408010104.csv',
 'race_data\\horse_info_202408010106.csv',
 'race_data\\horse_info_202408010107.csv',
 'race_data\\horse_info_202408010108.csv',
 'race_data\\horse_info_202408010109.csv',
 'race_data\\horse_info_202408010110.csv',
 'race_data\\horse_info_202408010111.csv',
 'race_data\\horse_info_202408010112.csv',
 'race_data\\horse_info_202408010201.csv',
 'race_data\\horse_info_202408010202.csv',
 'race_data\\horse_info_202408010205.csv',
 'race_data\\horse_info_202408010206.csv',
 'race_data\\horse_info_202408010207.csv',
 'race_data\\horse_info_202408010208.csv',
 'race_data\\horse_info_202408010209.csv',
 'race_data\\horse_info_202408010210.csv',
 'race_data\\horse_info_202408010211.csv',
 'race_data\\horse_info_202408010212.csv',
 'race_data\\horse_info_202408010301.csv',
 'race_data\\horse_info_202408010302.csv',
 'race_data

In [124]:
# デコレータの定義
def track_calls(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        wrapper.call_count += 1
        return func(*args, call_count=wrapper.call_count, **kwargs)
    wrapper.call_count = 0
    return wrapper

@track_calls
def load_processed_df(p_csv, call_count):
    df = pd.read_csv(p_csv)
    df.columns = [c.replace(' ', '') for c in df.columns]
    df = df[USE_COL]
    df["レースID"] = call_count  # 各DataFrameにレースIDを付与
    df['性'] = df['性齢'].map(lambda x: x[0])
    df['齢'] = df['性齢'].map(lambda x: int(x[1:]))
    df['馬体重'] = df['馬体重'].map(lambda s: s if s != '計不' else None)
    df['当日体重'] = df['馬体重'].map(lambda s: int(s.split('(')[0]) if s is not None else None)
    df['増減'] = df['馬体重'].map(lambda s: int(s.split('(')[1].replace(')', '')) if s is not None else None)
    df.loc[df['着順'] == 1, '着差'] = 0
    df = df.drop(['性齢', '馬体重'], axis=1)
    df = df.loc[~df['着順'].isin(['取', '中', '除', '失'])]
    return df

In [125]:
# 複数のDataFrameをリストとして保持
dfs = [load_processed_df(p_race_csv) for p_race_csv in p_race_csvs]

# レースIDを追加して1つのデータフレームに統合
for i, df in enumerate(dfs):
    df["レースID"] = i  # 各DataFrameにレースIDを付与

# 単一のデータフレームに統合
merged_df = pd.concat(dfs, ignore_index=True)

merged_df

,着順,馬名,斤量,騎手,タイム,着差,単勝,人気,調教師,horse_id,jockey_id,レースID,性,齢,当日体重,増減
0,1,セレブレイトエール,57.0,坂井瑠星,1:55.4,0,2.6,1.0,[西] 大久保龍,2021104109,1163,0,牡,3,522.0,-4.0
1,2,カフジテルビウム,57.0,長岡禎仁,1:55.9,3,14.7,5.0,[西] 杉山佳明,2021102150,1142,0,牡,3,464.0,4.0
2,3,ルージュシュエット,55.0,鮫島克駿,1:55.9,アタマ,32.3,9.0,[西] 矢作芳人,2021105640,1157,0,牝,3,442.0,-2.0
3,4,ヤマニンアラクリア,57.0,田中健,1:56.2,1.3/4,19.3,7.0,[西] 中村直也,2021100284,1114,0,牡,3,480.0,10.0
4,5,アメリカンチーフ,57.0,松若風馬,1:56.2,クビ,5.0,2.0,[西] 音無秀孝,2021110123,1154,0,牡,3,468.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7489,13,ロゼフレア,53.0,岩田望来,1:22.3,クビ,20.9,7.0,[西] 中村直也,2021103203,1174,564,牝,3,452.0,6.0
7490,14,アサギリ,52.0,永島まな,1:22.3,ハナ,68.7,14.0,[東] 尾形和幸,2021100610,1187,564,牝,3,460.0,-4.0
7491,15,エイムフォーエース,55.0,高杉吏麒,1:22.3,アタマ,21.7,8.0,[東] 武井亮,2021104167,1213,564,牡,3,444.0,0.0
7492,16,クイックバイオ,54.0,佐々木大,1:22.4,3/4,8.4,5.0,[西] 須貝尚介,2021105374,1197,564,牝,3,490.0,-4.0


In [126]:
# レースIDを取得
race_ids = merged_df["レースID"].unique()
train_ids, test_ids = train_test_split(race_ids, test_size=0.2, shuffle=False)
print(f'train size: {len(train_ids)}, test size: {len(test_ids)}')

# トレーニングデータとテストデータに分割
train_df = merged_df[merged_df["レースID"].isin(train_ids)]
test_df = merged_df[merged_df["レースID"].isin(test_ids)]

# グループ情報の再計算
train_group = train_df.groupby("レースID").size().tolist()
test_group = test_df.groupby("レースID").size().tolist()

# 特徴量とターゲットの抽出
features = ["斤量", "horse_id", "jockey_id", "人気", "当日体重", "増減"]
X_train = train_df[features]
X_test = test_df[features]
# 着順を逆符号にしてスコア化
y_train = 18 - train_df["着順"].astype(float)
y_test = 18 - test_df["着順"].astype(float)

train size: 452, test size: 113


In [127]:
# LightGBM データセットの作成
train_data = lgb.Dataset(X_train, label=y_train, group=train_group)
test_data = lgb.Dataset(X_test, label=y_test, group=test_group, reference=train_data)

# モデルのハイパーパラメータ設定
params = {
    "objective": "lambdarank",
    "metric": "ndcg",
    "boosting": "gbdt",
    "num_leaves": 31,
    "learning_rate": 0.05,
    "verbose": -1
}

# モデルの学習
model = lgb.train(params, train_data, valid_sets=[test_data], num_boost_round=100,)

In [128]:
# テストデータでの予測
y_pred = model.predict(X_test)

# NDCGスコアの計算
true_relevance = [list(y_test)]  # 着順をリストに変換（符号は逆転済み）
predicted_scores = [y_pred]
ndcg = ndcg_score(true_relevance, predicted_scores)
print(f"NDCGスコア: {ndcg}")

NDCGスコア: 0.969353810053077


In [130]:
test_df.loc[:, '予測'] = y_pred.copy()
race_ids = test_df.loc[:, 'レースID'].unique()
for race_id in race_ids:
    print(test_df.loc[test_df['レースID'] == race_id, ['着順', '予測', '馬名']].sort_values('予測', ascending=False))
    print('---')

      着順        予測         馬名
5958   2  1.240314  ルージュシークエル
5960   4  0.458577  ランフォースマイル
5957   1 -0.138091  フォーキャンドルズ
5967  11 -0.197845    ダノンカゼルタ
5965   9 -0.687288       ディニテ
5963   7 -0.723233    セイフウサツキ
5964   8 -0.897156       ツインギ
5959   3 -1.496025      パスコード
5962   6 -1.733568  マーゴットレジーナ
5961   5 -1.940905    ジェットエアー
5966  10 -2.059054    ホリゾンブルー
5969  13 -2.506134  ウインドラブハーツ
5968  12 -2.591971  キッシングセーラー
5970  14 -2.703877   インナーリソース
---
      着順        予測         馬名
5974   4  0.907059    サンジャシント
5973   3  0.897475      シームルグ
5983  13  0.351201      コンタンゴ
5977   7 -0.401558  ブラックソンソルト
5976   6 -0.722467     アイブリンク
5972   2 -1.101521    デリュージョン
5981  11 -1.436187     シエルルビー
5980  10 -1.492052      セイノスケ
5979   9 -1.979296   キーチファイター
5971   1 -2.160330  メイショウタマユラ
5978   8 -2.173045  ウォーターカルエル
5982  12 -2.294770     インピッシュ
5975   5 -2.313157  メイショウタザワコ
---
      着順        予測         馬名
5984   1  0.736787   ヒルノハンブルク
5990   7  0.351088  ランフォザプライド
5988   5  0.197988    サトノセーブル
59